In [2]:
import os
import sys
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from base64 import b64decode

script_dir = os.path.abspath("")
mymodule_dir = os.path.join(script_dir, "..", "scripts")
sys.path.append(mymodule_dir)
import cleaning

jobs = cleaning.prep(pd.read_pickle("../data/jobs.pkl"))
jobs.head()

,Titel,Unternehmen,Ort,Gehalt_min,Gehalt_max,JobID,Link,Datum,Teilzeit_Remote,Introduction,...,Contact,Vollzeit,Teilzeit,Zeit flexibel,Remote,Job Kategorie,Junior,Gehalt_min_yearly,Gehalt_max_yearly,Gehalt_durchschnitt_yearly
0,(Junior) Data Scientist (m/w/d),KMH GmbH,Köln,4032,5726,57650956,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,"Vollzeit, Teilzeit, Home Office möglich",Wir suchen in Voll- oder Teilzeit\nWir suchen ...,...,Wir freuen uns über deine Bewerbung!\n\n KMH G...,True,True,True,True,Data Scientist,True,48384,68712,58548.0
1,Junior Data Scientist (m/w/d),TUI Cruises GmbH,Hamburg,3548,5081,57605321,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,"Vollzeit, Home Office möglich",TUI Cruises ist einer der führenden Kreuzfahrt...,...,TUI Cruises GmbH \nHeidenkampsweg 58\n20097 Ha...,True,False,False,True,Data Scientist,True,42576,60972,51774.0
2,Junior Data Scientist (m/f/x),TUI Cruises GmbH,Hamburg,3548,5081,57605323,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,"Vollzeit, Home Office möglich",TUI Cruises is one of the leading cruise provi...,...,TUI Cruises GmbH \nHeidenkampsweg 58\n20097 Ha...,True,False,False,True,Data Scientist,True,42576,60972,51774.0
3,(Junior) Data Scientist / Data Analyst (w/m/d),1&1,"Karlsruhe, Montabaur, Maintal",3871,6129,57608864,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,"Vollzeit, Home Office möglich",Mit rund 15 Mio. Kundenverträgen und 3.200 Mit...,...,,True,False,False,True,Data Scientist,True,46452,73548,60000.0
4,Junior Data Scientist (m/w/d) – Risikomodelle,Sparkassen Rating und Risikosysteme GmbH,Berlin,3710,5161,57181289,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,"Vollzeit, Home Office möglich",Die Sparkassen Rating und Risikosysteme (SR) i...,...,Herr Emanuel Reitzenstein Tel. +49 30 20672-0 ...,True,False,False,True,Data Scientist,True,44520,61932,53226.0


In [2]:
ethik = jobs.loc[
    jobs["Description"].str.lower().str.contains("ethi")
    | jobs["Profile"].str.lower().str.contains("ethi"),
    :,
]

In [8]:
jobs[jobs["Description"].str.lower().str.contains("bias")]

,Titel,Unternehmen,Ort,Gehalt_min,Gehalt_max,JobID,Link,Datum,Teilzeit_Remote,Introduction,...,Contact,Vollzeit,Teilzeit,Zeit flexibel,Remote,Job Kategorie,Junior,Gehalt_min_yearly,Gehalt_max_yearly,Gehalt_durchschnitt_yearly
79,Research Scientist in Data and AI Model Auditi...,Bosch Gruppe,Renningen,5000,7500,57651346,https://www.stepstone.de/stellenangebote--rese...,2023-09-04,"Vollzeit, Home Office möglich",Do you want beneficial technologies being shap...,...,www.bosch.com/research\nhttps://www.bosch-ai.c...,True,False,False,True,Data Scientist,False,60000,90000,75000.0
692,Senior Data Analyst | Product | Advertising Se...,Otto (GmbH & Co KG),Hamburg,4758,6855,57528522,https://www.stepstone.de/stellenangebote--seni...,2023-09-04,"Vollzeit, Home Office möglich",Advertising ist eine der drei strategischen Sä...,...,Svenja Kreutzfeldt Junior HR Managerin +49 4...,True,False,False,True,Data Analyst,False,57096,82260,69678.0
9077,Demand Analyst (m/w/d),Groupe SEB Deutschland GmbH,Geislingen an der Steige,4113,6129,57938604,https://www.stepstone.de/stellenangebote--dema...,2023-10-29,Vollzeit,Die Groupe SEB ist weltweit mit mehr als 33.00...,...,,True,False,False,False,Data Analyst,False,49356,73548,61452.0
9089,Business Analyst Intern - Luxembourg,Amazon Europe Core,Luxembourg,<NA>,<NA>,57820443,https://www.stepstone.de/stellenangebote--busi...,2023-10-29,Vollzeit,An Amazon internship will provide you with an ...,...,,True,False,False,False,Data Analyst,False,<NA>,<NA>,<NA>
15336,Business Analyst Intern,Amazon Europe Core,Luxembourg,<NA>,<NA>,58192947,https://www.stepstone.de/stellenangebote--busi...,2023-12-06,Vollzeit,An Amazon internship will provide you with an ...,...,,True,False,False,False,Data Analyst,False,<NA>,<NA>,<NA>


# Junior Stellen

In [4]:
junior_data_science_old = pd.read_csv("../data/junior_data_science.csv", index_col=0)
junior_data_science_all = jobs.loc[(jobs["Junior"]) & (jobs["Job Kategorie"].isin(["Data Scientist", "Data Analyst"])), :]

In [6]:
junior_data_science_berlin = junior_data_science_all.loc[junior_data_science_all["Ort"] == "Berlin", :]
junior_data_science_berlin.head()

,Titel,Unternehmen,Ort,Gehalt_min,Gehalt_max,JobID,Link,Datum,Teilzeit_Remote,Introduction,...,Contact,Vollzeit,Teilzeit,Zeit flexibel,Remote,Job Kategorie,Junior,Gehalt_min_yearly,Gehalt_max_yearly,Gehalt_durchschnitt_yearly
4,Junior Data Scientist (m/w/d) – Risikomodelle,Sparkassen Rating und Risikosysteme GmbH,Berlin,3710,5161,57181289,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,"Vollzeit, Home Office möglich",Die Sparkassen Rating und Risikosysteme (SR) i...,...,Herr Emanuel Reitzenstein Tel. +49 30 20672-0 ...,True,False,False,True,Data Scientist,True,44520,61932,53226.0
96,Data/Business Analyst (m/w/d) (Junior),Esteve Pharmaceuticals GmbH,Berlin,3548,5242,57586900,https://www.stepstone.de/stellenangebote--data...,2023-09-04,"Vollzeit, Home Office möglich","Esteve ist ein internationales, innovationsget...",...,Als Teil der ESTEVE-Familie haben Sie die Gele...,True,False,False,True,Data Analyst,True,42576,62904,52740.0
1319,(Junior) Business Analyst (m/w/d),GBI Development GmbH,Berlin,3387,5000,57640524,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,"Vollzeit, Home Office möglich",\nSMARTments business sind komplett ausgestatt...,...,Dann freuen wir uns auf Deine aussagekräftige ...,True,False,False,True,Data Analyst,True,40644,60000,50322.0
6528,(Junior) Analyst Finanzkriminalität Bank m/w/x,HR UNIVERSAL GmbH,Berlin,3065,3871,57672610,https://www.stepstone.de/stellenangebote--juni...,2023-09-07,"Vollzeit, Home Office möglich",,...,Wenn Sie sich in dieser anspruchsvollen Aufgab...,True,False,False,True,Data Analyst,True,36780,46452,41616.0
8907,(Junior) Digital Analyst (m/w/d),Cornelsen Verlag GmbH,Berlin,3468,4839,57926332,https://www.stepstone.de/stellenangebote--juni...,2023-10-25,"Vollzeit, Home Office möglich",Die Cornelsen Gruppe mit Hauptsitz in Berlin z...,...,,True,False,False,True,Data Analyst,True,41616,58068,49842.0


In [7]:
jobs.loc[jobs["JobID"] == 57181289, "Link"]

4    https://www.stepstone.de/stellenangebote--juni...
Name: Link, dtype: object

In [7]:
def file_as_pdf(df_, suffix):
	options = FirefoxOptions()
	options.add_argument("--headless")

	with webdriver.Firefox(options=options) as driver:
		wait = WebDriverWait(driver, 20)

		first = True
		for i, row in df_.iterrows():
			link = row["Link"]
			JobID = row["JobID"]
			driver.get(link)
			if first:
				# Überprüfen, ob es einen Cookie-Banner gibt
				wait.until(
					lambda driver: len(
						driver.find_elements(
							By.CSS_SELECTOR,
							"#ccmgt_explicit_preferences.privacy-prompt-button.secondary-button.options-button",
						)
					)
					> 0
				)
				# Klicke ersten Teil des Cookie Banners
				driver.find_element(
					By.CSS_SELECTOR,
					"#ccmgt_explicit_preferences.privacy-prompt-button.secondary-button.options-button",
				).click()

				# Klicke zweiten Teil des Cookie Banners
				wait.until(
					lambda driver: len(
						driver.find_elements(
							By.CSS_SELECTOR,
							".secondary-button.ccmgt_reject_button",
						)
					)
					> 0
				)
				driver.find_element(
					By.CSS_SELECTOR, ".secondary-button.ccmgt_reject_button"
				).click()
				first = False

			if len(driver.find_elements(By.CLASS_NAME, "js-app-ld-ListingOfflineBlock")) == 0:
				pdf = b64decode(driver.print_page())

				with open(f"../output/anzeigen/{JobID}_{suffix}.pdf", 'wb') as f:
					f.write(pdf)
			else:
				df_.drop(i)

In [8]:
file_as_pdf(junior_data_science_berlin, "berlin")

The geckodriver version (0.33.0) detected in PATH at /usr/local/bin/geckodriver might not be compatible with the detected firefox version (121.0); currently, geckodriver 0.34.0 is recommended for firefox 121.*, so it is advised to delete the driver in PATH and retry


In [27]:
# look up specific job with id
jobs.loc[jobs["JobID"] == 58185276, ["Link", "Gehalt_min", "Gehalt_max"]]

,Link,Gehalt_min,Gehalt_max
13296,https://www.stepstone.de/stellenangebote--juni...,<NA>,<NA>
